**Data loader**

In [20]:
import os
import pandas as pd
from urllib import urlopen
import os

decklists_path = 'data/decks/'
matches_path = 'data/matches/'

loader_exists = False
converter_exists = False
saver_exists = False

class Loader():
    def __init__(self):
        loader_exists = True
        self.deckfiles = []
        self.decks = []
        self.cards = []
        self.player_archetype = []
        self.allcards = []
        self.alias_dict = {}
        
    def loadAllDeckFilenames(self):
        if len(self.deckfiles) == 0:
            tmp = os.listdir(decklists_path)
            deckfiles = []
            for i in tmp:
                deckfiles.append(i.replace('~', ''))
            self.deckfiles = list(set(deckfiles))
        
    def loadDecks(self, filename):
        data = pd.read_json('data/decks/'+filename)
        return data['cards']
    
    def loadDecksData(self, filename):
        data = pd.read_json('data/decks/'+filename)
        return data
    
    def loadPlayerArchetype(self, filename):
        data = pd.read_json('data/decks/'+filename)
        tmp = []
        for i in range(0, len(data)):
            tmp.append({'draft': data['draft'][i], 'player': data['player'][i], 'archetype': data['name'][i]})
        return tmp
            
    def loadAllPlayerArchetype(self):
        self.loadAllDeckFilenames()
        for i in self.deckfiles:
            tmp = self.loadPlayerArchetype(i)
            for j in tmp:
                self.player_archetype.append(j)
    
    def loadAllDecks(self):
        self.loadAllDeckFilenames()
        for i in self.deckfiles:
            tmp_decks = self.loadDecks(i)
            for d in tmp_decks:
                self.decks.append(d)
    
    def loadAllUsedCards(self):
        if len(self.cards) == 0:
            self.loadAllDecks()
            for i in self.decks:
                for j in i:
                    if "(ORI)" not in j:
                        self.cards.append(j)
            self.cards = sorted(list(set(self.cards)))
    
    def downloadAllCardsData(self):
        URL = 'https://mtgjson.com/json/AllCards.json.zip'
        url = urlopen(URL)
        output = open('zippedCards.zip', 'wb')
        output.write(url.read())
        output.close()
        os.system('unzip zippedCards.zip')
        os.remove('zippedCards.zip')
        self.allcards = pd.read_json('AllCards.json')
    
    def loadAllCardsData(self):
        if os.path.isfile('AllCards.json') == False:
            self.downloadAllCardsData()
        if len(self.allcards) == 0:
            self.allcards = pd.read_json('AllCards.json')
            
    def loadCurrentCubeCards(self, filename):
        self.cube = []
        self.cube_dict = {}
        my_file = open('data/'+filename, 'r')
        my_string = my_file.read()
        my_list = my_string.split('\r\n')
        for i in my_list:
            tmp = i.split(' ')
            if len(tmp) > 1:
                cardname = " ".join(tmp[1:])
                self.cube.append(cardname)
                self.cube_dict[tmp[0]] = cardname
            
    def findInvalidCardnames(self, cubefile):
        self.invalid_names = []
        self.loadAllCardsData()
        self.loadAllUsedCards()
        self.loadCurrentCubeCards(cubefile)
        for i in self.cards:
            if i not in self.allcards:
                self.invalid_names.append(i)
        for i in self.cube:
            if i not in self.allcards:
                self.invalid_names.append(i)
        
        print self.invalid_names
                
    def correctNamesInAllDecks(self, to_correct_file):
        self.loadAllDeckFilenames()
        self.findInvalidCardnames('cubes/numbered_18_6.txt')
        cards_to_correct = self.loadNamesToCorrect(to_correct_file)
        for deckfile in self.deckfiles:
            my_file = open('data/decks/'+deckfile, 'r')
            tmp = my_file.read()
            for j in self.invalid_names:
                tmp = tmp.replace(j, cards_to_correct[j])
            my_file.close()
            out_file = open('data/decks/'+deckfile, 'w')
            out_file.write(tmp)
            out_file.close()
            
    def correctNamesInAllDecksTxt(self, filenames, to_correct_file):
        self.findInvalidCardnames('cubes/numbered_18_6.txt')
        cards_to_correct = self.loadNamesToCorrect(to_correct_file)
        for deckfile in filenames:
            my_file = open('data/for_conversion/'+deckfile, 'r')
            tmp = my_file.read()
            for j in self.invalid_names:
                if j not in cards_to_correct.keys():
                    print j
                else:
                    tmp = tmp.replace(j, cards_to_correct[j])
            my_file.close()
            out_file = open('data/for_conversion/'+deckfile, 'w')
            out_file.write(tmp)
            out_file.close()
            
    def loadNamesToCorrect(self, filename):
        my_file = open('data/'+filename, 'r')
        my_string = my_file.read()
        my_string = my_string.replace('\r', '')
        my_list = my_string.split('\n')
        cards_to_correct = {}
        for i in my_list:
            tmp = i.split(':')
            cards_to_correct[tmp[0]] = tmp[1]
        return cards_to_correct
        
    def loadPlayerAliases(self):
        if len(self.alias_dict) == 0:
            my_file = open('data/player_alias.txt', 'r')
            my_string = my_file.read()
            my_list = my_string.split('\n')
            for i in my_list:
                if i != '':
                    tmp = i.split(' ')
                    self.alias_dict[tmp[0]] = tmp[1] 
        
        return self.alias_dict
    
    def loadMatchesFromTxt(self, filename):
        my_file = open('data/'+filename, 'r')
        my_string = my_file.read()
        my_string = my_string.replace('\r', '')
        my_list = my_string.split('\n')
        loaded_matches = []
        for i in my_list:
            if i != '':
                tmp = i.split(' ')
                loaded_matches.append(tmp)
        return loaded_matches
    
    def loadMatches(self, filename):
        matches = pd.read_json('data/matches/'+filename)
        match_data = []
        print matches["draft"]
        for i in matches["matches"]: 
            record = []
            record.append({"player": i[0]["player"], "score": i[0]["score"]})
            record.append({"player": i[1]["player"], "score": i[1]["score"]})
            match_data.append(record)
        return match_data
    
    def loadMatchDataFilenames(self):
        return os.listdir('data/matches/')
    
    def loadAllMatches(self):
        match_files = self.loadMatchDataFilenames()
        full_match_data = []
        for i in match_files:
            print(i)
            match_data = self.loadMatches(i)
    
    def loadDecksFromTxt(self, filename):
        my_file = open('data/'+filename, 'r')
        my_string = my_file.read()
        my_string = my_string.replace('\r', '')
        my_list = my_string.split('NEXT_DECK')
        decks = []
        for i in my_list:
            current_deck = {}
            tmp = i.split('\n')
            for j in tmp:
                dict_line = j.split(':')
                if dict_line[0] == 'player':
                    current_deck['player'] = dict_line[1]
                if dict_line[0] == 'name': 
                    current_deck['name'] = dict_line[1]
                if dict_line[0] == 'cards':
                    card_list = dict_line[1].split(' ')
                    current_deck['cards'] = card_list
                if dict_line[0] == 'basic':
                    basic_list = dict_line[1].split(' ')
                    current_deck['basic'] = basic_list
            decks.append(current_deck)
        return decks
    
    def loadDecksFromTxtCardnames(self, filename):
        my_file = open('data/for_conversion/'+filename, 'r')
        my_string = my_file.read()
        my_string = my_string.replace('\r', '')
        my_list = my_string.split('NEXT_DECK')
        decks = []
        for i in my_list:
            current_deck = {}
            tmp = i.split('\n')
            for j in tmp:
                dict_line = j.split(':')
                if dict_line[0] == 'player':
                    current_deck['player'] = dict_line[1]
                if dict_line[0] == 'name': 
                    current_deck['name'] = dict_line[1]
                if dict_line[0] == 'cards':
                    card_list = dict_line[1].split('|')
                    current_deck['cards'] = card_list
                if dict_line[0] == 'basic':
                    basic_list = dict_line[1].split(' ')
                    current_deck['basic'] = basic_list
            decks.append(current_deck)
        return decks
       
    def updateCubeList(self, input_filename, output_filename, change_filename):
        input_file = open('data/update/'+input_filename, 'r')
        input_str = input_file.read()
        changes = self.loadNamesToCorrect('/update/'+change_filename)
        
        for i in changes:
            input_str = input_str.replace(i, changes[i])
            
        output_file = open('data/update/'+output_filename, 'w')
        output_file.write(input_str)
        output_file.close()

loader = Loader()

#loader.loadAllUsedCards()

#cubefile = 'numbered_18_3.txt'

#loader.loadCurrentCubeCards(cubefile)

#decks = loader.loadDecks('decklists_18_3.json')

#print decks[0]

#print loader.cube_dict

#loader.findInvalidCardnames(cubefile)

#mdata = loader.loadMatches('matches_15_1.json')
#for i in mdata: print i

#loader.loadAllCardsData()

#print loader.allcards['Raging Goblin']

#loader.loadAllMatches()   
#loader.updateCubeList('numbered_18_7.txt', 'numbered_18_8.txt', 'changes_18_8.txt')

In [21]:
class Saver():
    def __init__(self):
        saver_exists = True
        pass
    
    def saveMatchesAsJson(self, matches_string, filename):
        my_file = open('data/matches/'+filename+'.json', 'w')
        my_file.write(matches_string)
        my_file.close()
        
    def saveMatchScoresForCoverage(self, prepared, filename):
        ftmp = filename.split('.')
        my_file = open('data/coverage/'+'coverage_'+ftmp[0]+'.txt', 'w')
        my_file.write(prepared)
        my_file.close()
        
    def saveDecklistsAsJson(self, filename, decklists_string):
        filename = filename.replace('.txt', '.json')
        my_file = open('data/decks/'+filename, 'w')
        my_file.write(decklists_string)
        my_file.close()
    def saveDecklistsForCoverage(self, filename, decks_string):
        filename = 'coverage_'+filename.replace('json', 'txt')
        my_file = open('data/coverage/'+filename, 'w')
        my_file.write(decks_string)
        my_file.close()
        
saver = Saver()

In [25]:
class Converter():
    def __init__(self):
        converter_exists = True
        if loader_exists == False:
            loader = Loader()
        if saver_exists == False:
            saver = Saver()
            
    def getNumberOfRounds(self, match_num):
        for i in range (1, match_num/2):
            if (i*(i+1))/2 == match_num:
                return i+1
    
    def convertMatchesToJson(self, filename):
        alias_dict = loader.loadPlayerAliases()
        matches_to_convert = loader.loadMatchesFromTxt(filename)
        ftmp = filename.split('.')
        draft_num = ftmp[0].replace('matches_', '')
        matches_json = '{\n'
        matches_json +='\t"draft":"'+draft_num+'",\n'
        matches_json +='\t"cube_version":"'+draft_num+'",\n'
        matches_json +='\t"matches":[\n'
        for i in range(0, len(matches_to_convert)):
            matches_json+= '\t\t[{"player":"'+alias_dict[matches_to_convert[i][0]]+'", "score":'+matches_to_convert[i][1]
            matches_json+= '}, {"player":"'+alias_dict[matches_to_convert[i][2]]+'", "score":'+matches_to_convert[i][3]
            if i < len(matches_to_convert)-1:
                matches_json+='}],\n'
            else: matches_json+='}]\n\t]\n'
        matches_json+='}'
        
        saver.saveMatchesAsJson(matches_json, ftmp[0])
        
    def prepareMatchesForCoverage(self, filename):
        match_data = loader.loadMatches(filename)
        match_num = len(match_data)
        draft_rounds = 3 #self.getNumberOfRounds(match_num)
        prepared = ''
        current_round = 0
        match_per_round = match_num / draft_rounds
        for i in range(0, match_num):
            if i%match_per_round == 0:
                current_round +=1
                prepared+='\nRunda #'+str(current_round)+'\n\n'
            prepared+=str(i+1)+'. '+match_data[i][0]['player']+' vs '+match_data[i][1]['player']+' '
            prepared+=str(match_data[i][0]['score'])+':'+str(match_data[i][1]['score'])+'\n'
        
        saver.saveMatchScoresForCoverage(prepared, filename)
        
    def convertNumbersToCardnames(self, numbers):
        cardnames = []
        for i in numbers:
            if i in loader.cube_dict:
                cardnames.append(loader.cube_dict[i])
        return cardnames
               
    def convertDecksTxtToJson(self, filename, cubefile):
        ftmp = filename.split('_')
        draft_num = ftmp[1]+'_'+ftmp[2].replace('.txt', '')
        decks = loader.loadDecksFromTxt(filename)
        # print(decks)
    
        loader.loadCurrentCubeCards(cubefile)
        json_string = '[\n'
        for i in range(0, len(decks)):
            json_string+='\t{\n\t\t"draft": "'+draft_num+'",\n'
            json_string+='\t\t"player": "'+decks[i]['player']+'",\n'
            json_string+='\t\t"name": "'+decks[i]['name']+'",\n'
            json_string+='\t\t"cards": ['
            cardnames = self.convertNumbersToCardnames(decks[i]['cards'])
            for j in range(0, len(cardnames)-1):
                json_string+='\n\t\t\t"'+cardnames[j]+'",'
            json_string+='\n\t\t\t"'+cardnames[len(cardnames)-1]+'"\n\t\t],'
            json_string+='\n\t\t"basic": {'
            json_string+='\n\t\t\t"Plains": '+str(decks[i]["basic"][0])+','
            json_string+='\n\t\t\t"Island": '+str(decks[i]["basic"][1])+','
            json_string+='\n\t\t\t"Swamp": '+str(decks[i]["basic"][2])+','
            json_string+='\n\t\t\t"Mountain":'+str(decks[i]["basic"][3])+','
            json_string+='\n\t\t\t"Forest": '+str(decks[i]["basic"][4])+'\n\t\t}'
            if i < len(decks) -1: 
                json_string+='\n\t},\n'
            else:
                json_string+='\n\t}\n'
        
        json_string+=']' 
        
        saver.saveDecklistsAsJson(filename, json_string)
        
    def convertDecksTxtToJsonCardnames(self, filename):
        ftmp = filename.split('_')
        draft_num = ftmp[1]+'_'+ftmp[2].replace('.txt', '')
        decks = loader.loadDecksFromTxtCardnames(filename)
        json_string = '[\n'
        for i in range(0, len(decks)):
            json_string+='\t{\n\t\t"draft": "'+draft_num+'",\n'
            json_string+='\t\t"player": "'+decks[i]['player']+'",\n'
            json_string+='\t\t"name": "'+decks[i]['name']+'",\n'
            json_string+='\t\t"cards": ['
            for j in range(0, len(decks[i]['cards'])-1):
                json_string+='\n\t\t\t"'+decks[i]['cards'][j]+'",'
            json_string+='\n\t\t\t"'+decks[i]['cards'][len(decks[i]['cards'])-1]+'"\n\t\t],'
            json_string+='\n\t\t"basic": {'
            json_string+='\n\t\t\t"Plains": '+str(decks[i]["basic"][0])+','
            json_string+='\n\t\t\t"Island": '+str(decks[i]["basic"][1])+','
            json_string+='\n\t\t\t"Swamp": '+str(decks[i]["basic"][2])+','
            json_string+='\n\t\t\t"Mountain":'+str(decks[i]["basic"][3])+','
            json_string+='\n\t\t\t"Forest": '+str(decks[i]["basic"][4])+'\n\t\t}'
            if i < len(decks) -1: 
                json_string+='\n\t},\n'
            else:
                json_string+='\n\t}\n'
        
        json_string+=']'
        
        saver.saveDecklistsAsJson(filename, json_string)
        
    def prepareDecklistsForCoverage(self, filename):
        data = loader.loadDecksData(filename)
        decks_string = ''
        for i in range(0, len(data['player'])):
            decks_string+='Player:  '+data['player'][i]+'\n'
            decks_string+='Archetype: '+data['name'][i]+'\n'
            decks_string+='--------------------\n'
            decks_string+='Decklist ('+str(len(data['cards'][i]))+'):\n'
            for j in data['cards'][i]:
                decks_string+=j+'\n'
            decks_string+='--------------------\n'
            basics = data['basic'][i]
            basic_count = basics['Plains']+basics['Island']+basics['Swamp']+basics['Mountain']+basics['Forest']
            decks_string+='Basic lands ('+str(basic_count)+')\n'
            if basics['Plains'] > 0: decks_string+=str(basics['Plains'])+' Plains\n'
            if basics['Island'] > 0: decks_string+=str(basics['Island'])+' Island\n'
            if basics['Swamp'] > 0: decks_string+=str(basics['Swamp'])+' Swamp\n'
            if basics['Mountain'] > 0: decks_string+=str(basics['Mountain'])+' Mountain\n'
            if basics['Forest'] > 0: decks_string+=str(basics['Forest'])+' Forest\n'
            if i < len(data['player']) -1: decks_string +='\n*++++++++++++++++++++*\n'
            
        saver.saveDecklistsForCoverage(filename, decks_string)
    
    def convertMatchDataToAliases(self, aliases_filename):
        match_files = loader.loadMatchDataFilenames()
        alias_dict = {}
        my_file = open('data/'+aliases_filename, 'r')
        atmp = my_file.read()
        alist = atmp.split('\n')
        alist = alist[:len(alist)-1]
        for i in alist:
            ltmp = i.split(' ')
            alias_dict[ltmp[0]] = ltmp[1]
        
        for i in match_files:
            my_file = open('data/matches/'+i, 'r')
            tmp = my_file.read()
            for j in alias_dict:
                tmp = tmp.replace(j, alias_dict[j])
            new_file = open('data/alias_matches/'+i, 'w')
            new_file.write(tmp)

        
converter = Converter()
converter.convertMatchesToJson('matches_18_11.txt')

converter.convertDecksTxtToJson('decklists_18_11.txt', 'numbered_18_11.txt')
converter.prepareMatchesForCoverage('matches_18_11.json')

converter.prepareDecklistsForCoverage('decklists_18_11.json')

#converter.convertMatchDataToAliases('name_with_alias.txt')

#loader.findInvalidCardnames('cubes/numbered_18_6.txt')

#loader.correctNamesInAllDecks('to_correct.txt')

#decknames = sorted(os.listdir('data/for_conversion/'))

#for i in decknames:
#    converter.convertDecksTxtToJsonCardnames(i)

0    18_11
1    18_11
2    18_11
3    18_11
4    18_11
5    18_11
Name: draft, dtype: object
